<a href="https://colab.research.google.com/github/nikitaoltyan/Mech-Trading-Systems/blob/main/stochastic_parabolic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Project cloning
!git clone https://github.com/nikitaoltyan/Mech-Trading-Systems.git
%cd Mech-Trading-Systems

Cloning into 'Mech-Trading-Systems'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 297 (delta 18), reused 8 (delta 3), pack-reused 262
Receiving objects: 100% (297/297), 1.97 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/Mech-Trading-Systems


In [2]:
import pandas as pd
from PlotDraw import draw_plot

In [3]:
data = pd.read_excel("LHAG.xlsx")
data.head()

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M


In [4]:
data.astype({'<CLOSE>': 'int32', '<OPEN>': 'int32', '<HIGH>': 'int32', '<LOW>': 'int32'}).dtypes
data.astype({'<DATE>': 'datetime64[ns]'}).dtypes

<DATE>     datetime64[ns]
<CLOSE>           float64
<OPEN>            float64
<HIGH>            float64
<LOW>             float64
<VOL>              object
dtype: object

stochastic

In [8]:
def stochastic(data, period = 14, m = 3):
    stoch_data = data.copy()
    stoch_data['14-HIGH'] = stoch_data['<HIGH>'].rolling(period).max()
    stoch_data['14-LOW'] = stoch_data['<LOW>'].rolling(period).min()
    stoch_data['%K'] = (stoch_data['<CLOSE>'] - stoch_data['14-LOW'])*100/(stoch_data['14-HIGH'] - stoch_data['14-LOW'])
    stoch_data['%D'] = stoch_data['%K'].rolling(m).mean()
    return stoch_data

In [10]:
stoch_data = stochastic(data)
stoch_data

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>,14-HIGH,14-LOW,%K,%D
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M,NaN,NaN,NaN,NaN
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M,NaN,NaN,NaN,NaN
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M,NaN,NaN,NaN,NaN
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M,NaN,NaN,NaN,NaN
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1258,"Jan 08, 2016",14.920,15.060,15.140,14.840,5.90M,15.14,13.205,88.630491,79.196200
1259,"Jan 07, 2016",15.010,14.800,15.150,14.615,8.07M,15.15,13.205,92.802057,87.479378
1260,"Jan 06, 2016",15.160,14.910,15.410,14.900,8.04M,15.41,13.205,88.662132,90.031560
1261,"Jan 05, 2016",14.925,14.670,14.995,14.625,6.52M,15.41,13.205,78.004535,86.489574


In [18]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [31]:
def plot_stochastic(dataframe, start = 1, end = len(data)):
  fig = make_subplots(rows=2, cols=1)
  fig.add_trace(go.Candlestick(x=[i for i in range (0, len(dataframe))][start:end],
                open=data['<OPEN>'][start:end],
                high=data['<HIGH>'][start:end],
                low=data['<LOW>'][start:end],
                close=data['<CLOSE>'][start:end]), 
                row = 1, col = 1)
  fig.add_trace(go.Scatter(x=[i for i in range (0, len(dataframe))][start:end], y=dataframe['%K'][start:end], line_color='#242bff'),
                row=2, col=1)
  fig.add_trace(go.Scatter(x=[i for i in range (0, len(dataframe))][start:end], y=dataframe['%D'][start:end], line_color='#ff6d24'),
                row=2, col=1)
  
  fig.update_layout(xaxis_rangeslider_visible=False)
  fig.show()

In [28]:
plot_stochastic(dataframe = stoch_data, start = 100, end = 300)

parabolic

In [13]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

Selecting previously unselected package libta-lib0.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 4.9MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp37-cp37m-linux_x86_64.whl size=1437978 sha256=00430b0eca445eb626ddfa4e1b6413fabcd28a73c94418c02d03aa03fa411218
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [15]:
dataSAR = data.copy()
dataSAR['SAR'] = talib.SAR(dataSAR['<HIGH>'], dataSAR['<LOW>'], acceleration=0.02, maximum=0.2)
dataSAR

,<DATE>,<CLOSE>,<OPEN>,<HIGH>,<LOW>,<VOL>,SAR
0,"Dec 30, 2020",10.815,11.000,11.030,10.735,3.15M,NaN
1,"Dec 29, 2020",10.960,10.630,11.290,10.560,11.25M,11.290000
2,"Dec 28, 2020",10.375,10.170,10.615,10.155,6.45M,11.290000
3,"Dec 23, 2020",9.860,9.518,9.888,9.496,3.55M,11.290000
4,"Dec 22, 2020",9.450,9.390,9.600,9.328,3.16M,11.182360
...,...,...,...,...,...,...,...
1258,"Jan 08, 2016",14.920,15.060,15.140,14.840,5.90M,13.402016
1259,"Jan 07, 2016",15.010,14.800,15.150,14.615,8.07M,13.541055
1260,"Jan 06, 2016",15.160,14.910,15.410,14.900,8.04M,13.701949
1261,"Jan 05, 2016",14.925,14.670,14.995,14.625,6.52M,13.906915


In [32]:
def plot_sar(dataframe, start = 1, end = len(data)):
  fig = make_subplots(rows=2, cols=1)
  fig.add_trace(go.Candlestick(x=[i for i in range (0, len(dataframe))][start:end],
                open=data['<OPEN>'][start:end],
                high=data['<HIGH>'][start:end],
                low=data['<LOW>'][start:end],
                close=data['<CLOSE>'][start:end]), 
                row = 1, col = 1)
  fig.add_trace(go.Scatter(x = [i for i in range (0, len(data))][start:end] , y = dataframe['SAR'][start:end], line = go.scatter.Line(color="gray")))
  fig.show()

In [35]:
plot_sar(dataframe = dataSAR, start = 100, end = 300)